# Iterated Watersheds

In this notebook we aim to present a few results using the iterated watersheds algorithm. Please refer to the article for detailed explanation.

All the helper functions are given in the file utils_IteratedWatersheds.py. See this for exact details of implementation. The datasets can be downloaded from the from the following links

[1] Weizman Dataset - http://www.wisdom.weizmann.ac.il/~vision/Seg_Evaluation_DB/

[2] Road Network Dataset - https://figshare.com/articles/Urban_Road_Network_Data/2061897

In [14]:
from IPython.display import HTML
print("The raw code to generate all the results can be hidden/shown by clicking the below button!")
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

The raw code to generate all the results can be hidden/shown by clicking the below button!


In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import time

%matplotlib inline

from utils_IteratedWatersheds import *

## Results on Weizman 1-Object and 2-Object Datasets

To visualize the results of the Iterated Watersheds, we use the image segmentation database for better understanding. Since, image segmentation databases have labelled ground truth, it makes it easier to validate a new algorithm. Here, we compare the Iterated Watersheds with Spectral clustering, Isoperimetric Partitioning and simple K-Means as well.

The following tables, display the results.

In [ ]:
# Code to generate the results
generate_data = generate_data_1Object()

with open("./results.csv", 'w') as f:
    f.write("id,method,img_name,parameters,AMI,ARI,F,accuracy,time")
    f.write("\n")
    count = 0
    for img, gt, name in generate_data:

        # Construct the Graph
        s0, s1, s2 = img.shape
        graph = img_to_graph(img, beta=5., which='dissimilarity')
        xgrid, ygrid = np.meshgrid(np.arange(s0), np.arange(s1))
        X = (np.vstack((xgrid.flatten(), ygrid.flatten())).transpose())

        # Iterated Watersheds
        for nClust in [2]:
            for rep in range(20):
                tic = time.time()
                labels_IW, cost_history, centers = iterated_watershed(graph, X, number_clusters=nClust, max_iterations = 20 )
                time_measure = time.time() - tic
                AMI, ARI, F, acc = evaluate_output(labels_IW, gt)
                param = "nCluster="+str(nClust)+"repeat="+str(rep)
                f.write("{},{},{},{},{},{},{},{},{}\n".format(count,"IteratedWatershed",name,param,AMI,ARI,F,acc,time_measure))
                count += 1

        # Spectral Clustering
        for nClust in [2]:
            for beta in [1.,2.,3.,5.]:
                tic = time.time()
                labels_spectral = spectral_clustering(graph, n_clusters=nClust, beta_weight=beta, eps_weight=1e-6)
                time_measure = time.time() - tic
                AMI, ARI, F, acc = evaluate_output(labels_spectral, gt)
                param = "nCluster="+str(nClust)+"beta="+str(beta)
                f.write("{},{},{},{},{},{},{},{},{}\n".format(count,"SpectralClustering",name,param,AMI,ARI,F,acc,time_measure))
                count += 1

        # Isoperimetric Partitioning - Full
        for beta in [1.,2., 3., 5.]:
            tic = time.time()
            labels_isoFull = isoperimetric_partitioning(graph, beta_weight=beta, eps_weight=1e-6,  which='full')
            time_measure = time.time() - tic
            AMI, ARI, F, acc = evaluate_output(labels_isoFull, gt)
            param = "beta="+str(beta)
            f.write("{},{},{},{},{},{},{},{},{}\n".format(count,"IsoFull",name,param,AMI,ARI,F,acc,time_measure))
            count += 1

        # Isoperimetric Partitioning - Recursive
        for beta in [1.,2., 3., 5.]:
            tic = time.time()
            labels_isoRecursive = isoperimetric_partitioning(graph, beta_weight=beta, eps_weight=1e-6,  which='recursive')
            time_measure = time.time() - tic
            AMI, ARI, F, acc = evaluate_output(labels_isoRecursive, gt)
            param = "beta="+str(beta)
            f.write("{},{},{},{},{},{},{},{},{}\n".format(count,"IsoRecursive",name,param,AMI,ARI,F,acc,time_measure))
            count += 1

        # K-Means (Using img)
        for nClust in [2]:
            tic = time.time()
            labels_kmeans = kmeans_adapted(img, n_clusters=nClust)
            time_measure = time.time() - tic
            AMI, ARI, F, acc = evaluate_output(labels_kmeans, gt)
            param = "nClust="+str(nClust)
            f.write("{},{},{},{},{},{},{},{},{}\n".format(count,"KMeans",name,param,AMI,ARI,F,acc,time_measure))
            count += 1

In [ ]:
generate_data = generate_data_2Object()

def _write(string):
    with open("./results_2Obj.csv", 'a') as f:
        f.write(string)


with open("./results_2Obj.csv", 'w') as f:
    f.write("id,method,img_name,parameters,AMI,ARI,F,accuracy,time\n")

processes = []
for img, gt, name in generate_data:
    s0, s1, s2 = img.shape
    graph = img_to_graph(img, beta=5., which='dissimilarity')
    xgrid, ygrid = np.meshgrid(np.arange(s0), np.arange(s1))
    X = (np.vstack((xgrid.flatten(), ygrid.flatten())).transpose())
    count = 0
    # Iterated Watersheds
    for nClust in [3]:
        for rep in range(20):
            tic = time.time()
            labels_IW, cost_history, centers = iterated_watershed(graph, X, number_clusters=nClust, max_iterations = 20 )
            time_measure = time.time() - tic
            AMI, ARI, F, acc = evaluate_output(labels_IW, gt)
            param = "nCluster="+str(nClust)+"repeat="+str(rep)
            _write("{},{},{},{},{},{},{},{},{}\n".format(count,"IteratedWatershed",name,param,AMI,ARI,F,acc,time_measure))
            count += 1

    # Spectral Clustering
    for nClust in [3]:
        for beta in [1.,2.,3.,5.]:
            tic = time.time()
            labels_spectral = spectral_clustering(graph, n_clusters=nClust, beta_weight=beta, eps_weight=1e-6)
            time_measure = time.time() - tic
            AMI, ARI, F, acc = evaluate_output(labels_spectral, gt)
            param = "nCluster="+str(nClust)+"beta="+str(beta)
            _write("{},{},{},{},{},{},{},{},{}\n".format(count,"SpectralClustering",name,param,AMI,ARI,F,acc,time_measure))
            count += 1

    # Isoperimetric Partitioning - Full
    for beta in [1.,2., 3., 5.]:
        tic = time.time()
        labels_isoFull = isoperimetric_partitioning(graph, beta_weight=beta, eps_weight=1e-6,  which='full')
        time_measure = time.time() - tic
        AMI, ARI, F, acc = evaluate_output(labels_isoFull, gt)
        param = "beta="+str(beta)
        _write("{},{},{},{},{},{},{},{},{}\n".format(count,"IsoFull",name,param,AMI,ARI,F,acc,time_measure))
        count += 1

    # Isoperimetric Partitioning - Recursive
    for beta in [1.,2., 3., 5.]:
        tic = time.time()
        labels_isoRecursive = isoperimetric_partitioning(graph, beta_weight=beta, eps_weight=1e-6,  which='recursive')
        time_measure = time.time() - tic
        AMI, ARI, F, acc = evaluate_output(labels_isoRecursive, gt)
        param = "beta="+str(beta)
        _write("{},{},{},{},{},{},{},{},{}\n".format(count,"IsoRecursive",name,param,AMI,ARI,F,acc,time_measure))
        count += 1

    # K-Means (Using img)
    for nClust in [3]:
        tic = time.time()
        labels_kmeans = kmeans_adapted(img, n_clusters=nClust)
        time_measure = time.time() - tic
        AMI, ARI, F, acc = evaluate_output(labels_kmeans, gt)
        param = "nClust="+str(nClust)
        _write("{},{},{},{},{},{},{},{},{}\n".format(count,"KMeans",name,param,AMI,ARI,F,acc,time_measure))
        count += 1
    


print("Done..")


In [9]:
# Code to display the table

import pandas as pd
import numpy as np
import tabulate

colnames = "id,method,img_name,parameters,AMI,ARI,F,accuracy,time".split(',')
data = pd.read_csv("./results_1Obj.csv", names=colnames)

dict_answer = {}
for method in ['IteratedWatershed', 'SpectralClustering', 'IsoFull', 'IsoRecursive', 'KMeans']:
    for measure in ['AMI', 'ARI', 'F', 'accuracy']:
        dict_answer[method+measure] = []
        

for img in np.unique(data['img_name']):
    for method in ['IteratedWatershed', 'SpectralClustering', 'IsoFull', 'IsoRecursive', 'KMeans']:
        for measure in ['AMI', 'ARI', 'F', 'accuracy']:
            ind = np.logical_and(data['method'] == method, data['img_name'] == img)
            val = float(np.max(data[ind][measure]))
            dict_answer[method+measure].append(val)
            

print("---------------------------------")
print("   Weizman 1-Object Dataset")
print("---------------------------------")
l = []
l.append(["Method", "AMI", "ARI", "F", "accuracy"])
for method in ['IteratedWatershed', 'SpectralClustering', 'IsoFull', 'KMeans']:
    tmp = [method]
    for measure in ['AMI', 'ARI', 'F', 'accuracy']:
        m = np.nanmean(dict_answer[method+measure])
        tmp.append("{:0.4f}".format(m))
    l.append(tmp)
display(HTML(tabulate.tabulate(l, tablefmt='html')))


colnames = "id,method,img_name,parameters,AMI,ARI,F,accuracy,time".split(',')
data = pd.read_csv("./results_2Obj.csv", names=colnames)

dict_answer = {}
for method in ['IteratedWatershed', 'SpectralClustering', 'IsoFull', 'IsoRecursive', 'KMeans']:
    for measure in ['AMI', 'ARI', 'F', 'accuracy']:
        dict_answer[method+measure] = []
        

for img in np.unique(data['img_name']):
    for method in ['IteratedWatershed', 'SpectralClustering', 'IsoFull', 'IsoRecursive', 'KMeans']:
        for measure in ['AMI', 'ARI', 'F', 'accuracy']:
            ind = np.logical_and(data['method'] == method, data['img_name'] == img)
            val = float(np.max(data[ind][measure]))
            dict_answer[method+measure].append(val)
            


print("---------------------------------")
print("   Weizman 2-Object Dataset")
print("---------------------------------")
l = []
l.append(["Method", "AMI", "ARI", "F", "accuracy"])
for method in ['IteratedWatershed', 'SpectralClustering', 'KMeans']:
    tmp = [method]
    for measure in ['AMI', 'ARI', 'F', 'accuracy']:
        m = np.nanmean(dict_answer[method+measure])
        tmp.append("{:0.4f}".format(m))
    l.append(tmp)
display(HTML(tabulate.tabulate(l, tablefmt='html')))


---------------------------------
   Weizman 1-Object Dataset
---------------------------------


Method,AMI,ARI,F,accuracy
IteratedWatershed,0.2467,0.3126,0.7880,0.8329
SpectralClustering,0.1674,0.1568,0.6889,0.8697
IsoFull,0.0712,0.0600,0.7772,0.7666
KMeans,0.1811,0.2043,0.6684,0.8143


---------------------------------
   Weizman 2-Object Dataset
---------------------------------


Method,AMI,ARI,F,accuracy
IteratedWatershed,0.3599,0.3615,0.7497,0.8974
SpectralClustering,0.2716,0.2635,0.7576,0.8963
KMeans,0.2417,0.2128,0.5893,0.8884


## Results on Road Network Dataset

In [ ]:
# Code to generate the results. Takes aroung 18 hours to generate the results
import multiprocessing as mp
import time

tic = time.time()
def _write(string):
    with open("./results_RoadNetwork.csv", 'a') as f:
        f.write(string)
    

with open("./results_RoadNetwork.csv", 'w') as f:
    f.write("id,city,method,nClust,rep,cost\n")

processes = []    
for city in ['Mumbai', 'Hyderabad', 'Chennai', 'Bengaluru', 'Calcutta', 'Delhi']:
    X, G = get_road_network_data(city)
    for nClust in [3,6,9,12,15]:
        for rep in range(30):
            # Iterated Watershed
            count = 0
            labels, cost_history, centers = iterated_watershed(G, X, number_clusters=nClust, max_iterations=30)
            _write("{},{},{},{},{},{}\n".format(count,city,'IteratedWatershed',nClust,rep,np.min(cost_history)))
            count += 1

            # K-Means
            cost_kmeans, labels_kmeans = kmeans_on_roadNetwork(G, X,  nClusters=nClust)
            _write("{},{},{},{},{},{}\n".format(count,city,'KMeans',nClust,rep,cost_kmeans))
            count += 1

            # Greedy K-Center
            cost_kCenter, labels_kCenter = greedy_kCenter(G, X, n_clusters=nClust)
            _write("{},{},{},{},{},{}\n".format(count,city,'KCenters',nClust,rep,cost_kCenter))

    
    print("Done in {} seconds....".format(time.time()-tic))

In [13]:
import pandas as pd
import numpy as np

colnames = "id,city,method,nClust,rep,cost".split(",")
data = pd.read_csv("./results_RoadNetwork.csv")
dict_answer = {}
for city in ['Mumbai', 'Hyderabad', 'Chennai', 'Bengaluru', 'Calcutta', 'Delhi']:
    for nClust in [3,6,9,12,15]:
        for method in ["IteratedWatershed", "KMeans", "KCenters"]:
            ind = np.logical_and(np.logical_and(data['city']==city, data['nClust']==nClust), data['method']==method)
            dict_answer[city+str(nClust)+method] = float(np.nanmean(data[ind]['cost']))
        a1, a2 = dict_answer[city+str(nClust)+"IteratedWatershed"], dict_answer[city+str(nClust)+"KMeans"]
        a3 = dict_answer[city+str(nClust)+"KCenters"]
        dict_answer[city+str(nClust)+"percent"+"KMeans"] = (a2 - a1)/a2
        dict_answer[city+str(nClust)+"percent"+"KCenters"] = (a3 - a1)/a3
        
# print("City& Number Clusters& Iterated Watershed & KMeans & \% Improvement & KCenters & \% Improvement \\\\ \\hline")            
for city in ['Mumbai', 'Hyderabad', 'Chennai', 'Bengaluru', 'Calcutta', 'Delhi']:
    print("-------------------------------")
    print("      ", city, "   ")
    print("-------------------------------")
    l = [["Number of Clusters", "Iterated Watershed", "KMeans", "% Improvement", "KCenters", "%Improvement"]]
    for nClust in [3,6,9,12,15]:
        tmp = [str(nClust)]
        for method in ["IteratedWatershed", "KMeans","KCenters"]:
            m = dict_answer[city+str(nClust)+method]
            tmp.append("{:0.2f}".format(m))
            if method in ["KMeans","KCenters"]:
                m = dict_answer[city+str(nClust)+"percent"+method]*100
                tmp.append("{:0.2f}".format(m))
        l.append(tmp)
    display(HTML(tabulate.tabulate(l, tablefmt='html')))            


-------------------------------
       Mumbai    
-------------------------------


Number of Clusters,Iterated Watershed,KMeans,% Improvement,KCenters,%Improvement
3,75625.21,85714.67,11.77,129256.23,41.49
6,46552.73,64684.32,28.03,96220.56,51.62
9,36549.42,53381.05,31.53,80014.19,54.32
12,31439.17,45545.34,30.97,69879.96,55.01
15,28973.69,40430.70,28.34,65174.44,55.54


-------------------------------
       Hyderabad    
-------------------------------


Number of Clusters,Iterated Watershed,KMeans,% Improvement,KCenters,%Improvement
3,70422.44,98289.47,28.35,150161.85,53.10
6,55801.81,63029.52,11.47,143362.81,61.08
9,47806.78,51303.10,6.82,139208.10,65.66
12,42282.13,52597.76,19.61,122434.80,65.47
15,38264.49,46480.52,17.68,111933.23,65.81


-------------------------------
       Chennai    
-------------------------------


...

Number of Clusters,Iterated Watershed,KMeans,% Improvement,KCenters,%Improvement
3,125300.14,219508.02,42.92,241052.79,48.02
6,91360.04,98072.17,6.84,183065.39,50.09
9,76429.72,83139.10,8.07,191371.54,60.06
12,65603.19,79167.20,17.13,166675.36,60.64
15,57498.55,67810.40,15.21,160847.42,64.25


-------------------------------
       Calcutta    
-------------------------------


Number of Clusters,Iterated Watershed,KMeans,% Improvement,KCenters,%Improvement
3,22084.11,24136.89,8.50,58609.46,62.32
6,15419.97,16741.95,7.90,58155.98,73.49
9,13414.80,14720.62,8.87,50962.84,73.68
12,11541.40,13365.42,13.65,51258.80,77.48
15,10297.36,12508.15,17.67,47098.97,78.14


-------------------------------
       Delhi    
-------------------------------


Number of Clusters,Iterated Watershed,KMeans,% Improvement,KCenters,%Improvement
3,60419.45,66272.17,8.83,116508.68,48.14
6,43123.33,51899.35,16.91,86064.13,49.89
9,34500.10,44890.19,23.15,74177.07,53.49
12,29953.21,34897.00,14.17,70639.21,57.60
15,26657.97,35593.96,25.11,63284.28,57.88
